In [3]:
#######################################################################################
## Script written by Gabrielle G. Jones and edited by Schuyler R. Borges. 
## This script was written to mix three planetary refelectance spectra simulated using 
## SMART at clear sky, high cloud, and low cloud conditions. Mixing these three spectra
## at different percentages produces planetary reflectance spectra with varying cloud
## cover (25%, 50%, 75%). This script is written for mixing planetary spectra of 
## one microbial surface at a time.
#######################################################################################

#######################################################################################
## List of definition functions to run first
#######################################################################################

# Definition function that takes a .rad file from SMART and calculates the reflectance 
# by dividing the radiance from the flux. The output of the definition function are reflectance 
# and radiance values for an entire column of data.
def getData(data):     # Input data
    
    columnfour = []    # Builds empty list for column four, radiance, in .rad file
    columnthree = []   # Builds empty list for column three, flux, in .rad file
    reflectance = []   # Builds empty list for reflectance values
    
    size = len(data)   # Defines size as length of the data file
   
    for i in range (size):                     
        columnfour.append(float(data[i][3]))   # Appends values from column four to empty list
        columnthree.append(float(data[i][2]))  # Appends values from column three to empty list
        
    f=np.array(columnfour)   # Builds array from column four list
    t=np.array(columnthree)  # Builds array from column three list
    
    reflectance=f/t   # Calculates reflectance values by dividing values from f by t
    
    return reflectance, f    # When calling definition function, it will return two values: the reflectance and radiance


# Definition function that organizes columns of values for exporting as a .txt file
def writetofile(filename, wave, flux, ref, rad):   # Input name of file, wavelength, flux, reflectance, and radiance values
    
    f = open(filename, 'w')                        # Creates file to write to
    
    for i in range(len(wave)):
        string = "%g   %g   %g   %g" % (wave[i], flux[i], ref[i], rad[i])   # For every wavelength value, create columns of wavelength,
        f.write(string + "\n")                                              # flux, reflectance, and radiance values
    
    f.close()    # write values to file, f, and close the file
    

# Definition function ensuring the length of a column in the array is the same as the length of the array
def getsize(arraytosize):                   # Input an array of lists
    minsize = len(arraytosize[0])           # Minsize is equal to the length of the first list in arraytosize
    print("Size in method: %d" % minsize)   # Print minsize
    
    for ar in arraytosize:                  
        sizeofar = len(ar)                  # Sizeofar is equal to the number of lists in the array, arraytosize
        if(sizeofar < minsize):             # If the length of arraytosize is less than the length of the first list,
            minsize = sizeofar              # the length of the first list is equal to the length of the array
            
    return minsize                          # Return minsize, which is the length of the first list in an array


# Definition function used to create mixed percentages of cloud spectra from SMART using defintion function above
def mixthemup(allReflect, allRad, percentage):              # Input an array of reflectance, radiance, and percentage values
    
    mixedReflect = [0] * getsize(allReflect)                # mixedReflect is equal to the first element of the first list of allReflect
    mixedRad = [0] * getsize(allRad)                        # mixedRad is equal to the first element of the first list of allRad
    
    print("Size of refarray: %d" % len(mixedReflect))       # Print length of mixedReflect
    print("Size of radarray: %d" % len(mixedRad))           # Print length of mixedRad
    
    k = 0                                                   
    
    for ref in allReflect:                      
        sizein = len(ref)                       # sizein is equal to the length of lists in the array, allReflect
        print("size fo ref: %d" % sizein)       # Print sizein
        
        # for each value in the file
        for j in range(sizein):                 # For each element in sizein
            
            # multiply the value by the corresponding %
            # add the result to a designated index for a given wavelength value
            
            mixedReflect[j] += float(ref[j]) * float(percentage[k]);
    
        k = k + 1
        
    k = 0
    
    for rad in allRad:                          
        sizein = len(rad)                       # sizein is equal to the length of lists in the array, allRad
    
        # for each value in the file
        for j in range(sizein):                 # For each element in sizein
            
            # multiply the value by the corresponding %
            # add the result to a designated index for a given wavelength value

            mixedRad[j] += float(rad[j]) * float(percentage[k]);
    
        k = k + 1
        
    return np.array(mixedReflect), np.array(mixedRad)     # Return an array of reflectance values and an array of radiance values

In [11]:
#######################################################################################
## Reading in data for one microbe at all cloud cover scenarios
#######################################################################################

# Import packages for python script
import matplotlib.pyplot as plt 
import os, glob
import numpy as np 
from astropy.io import ascii 
from astropy.table import Table
from astropy.table import Column

os.chdir('[enter path here]')               # Defines path to folder with SMART .rad files for mixing

files = glob.glob('File_names*.rad')        # Reads in .rad files from SMART for one organism and cloud scenario
print(files)                                # File_names should be a keyword that indicates each SMART file you
                                            # want to mix. This script is only written for individual microbes, so
                                            # an example keyword would be Arthrobacter, one of the microbes


data = ascii.read(files[0], data_start=0, delimiter='\s')   # Read in the first file in files

columnone = []          # Builds empty list for column one, wavelength, in .rad file
columnthree = []        # Builds empty list for column three, flux, in .rad file
wav = []                # Builds empty list for wavelength values
flux = []               # Builds empty list for flux values
size = len(data)        # Defines size as the length of the dataset
    
for i in range(size):
        
    columnone.append(float(data[i][0]))         # Appends values from column one to empty list
    columnthree.append(float(data[i][2]))       # Appends values from column three to empty list

wave = np.array(columnone)          # Defines wave as an array of wavelength values
flux = np.array(columnthree)        # Defines flux as an array of flux values

['Arthro_clearSky_scrubbed_updated_mon_toa.rad', 'Arthro_clearSky_um_mon_toa.rad', 'Arthro_highcloud_scrubbed_um_toa_new.rad', 'Arthro_highcloud_um_toa.rad', 'Arthro_lowcloud_scrub_um_toa_new.rad', 'Arthro_lowcloud_um_mon_toa.rad']


In [12]:
#######################################################################################
## Use the definition function, getData, to return one wavelength and then the 
## reflectance for each cloud condition for all the files
#######################################################################################

# Read in each individual file in files in ascii format
clearsky_scrub = ascii.read(files[0], data_start=0, delimiter='\s')     # Read in first .rad file associated with clear skies
                                                                        # without biosignatures

clearsky = ascii.read(files[1], data_start=0, delimiter='\s')           # Read in second .rad file associated with clear skies
                                                                        # with biosignatures

highcloud_scrub = ascii.read(files[2], data_start=0, delimiter='\s')    # Read in third .rad file associated with high cloud cover 
                                                                        # without biosignatures

highcloud = ascii.read(files[3], data_start=0, delimiter='\s')          # Read in fourth .rad file associated with high cloud cover 
                                                                        # with biosignatures

lowcloud_scrub = ascii.read(files[4], data_start=0, delimiter='\s')     # Read in fifth .rad file associated with low cloud cover
                                                                        # without biosignatures

lowcloud = ascii.read(files[5], data_start=0, delimiter='\s')           # Read in sixth .rad file associated with low cloud cover
                                                                        # with biosignatures

# Call definition function and define the reflectance (ref) and radiance (rad) outputs for each .rad file at
# varying cloud conditions (cs = clear skies, hc = high cloud, lc = low cloud)
cs_s_ref, cs_s_rad = getData(clearsky_scrub)            
cs_ref, cs_rad = getData(clearsky)                   
hc_s_ref, hc_s_rad = getData(highcloud_scrub)
hc_ref, hc_rad = getData(highcloud)
lc_s_ref, lc_s_rad = getData(lowcloud_scrub)
lc_ref, lc_rad = getData(lowcloud)

done1
done2
done3
done4
done5
done6


In [14]:
#######################################################################################
## Create a sum wavelength array that sums the reflectance of the all reflect at an
## index for each file in the reflectance data
#######################################################################################

# Import two additional packages for following script
import copy
import array as arr

# Define reflectance (ref) and radiance (rad) arrays for .rad files with biosignatures (originalarray)
# and without biosignatures (scrubbedarray) for each cloud condition (cs = clear skies, hc = high cloud,
# lc = low cloud)
orginalarray = [cs_ref, hc_ref, lc_ref]
orginalarray_rad = [cs_rad, hc_rad, lc_rad]
scrubbedarray = [cs_s_ref, hc_s_ref, lc_s_ref]
scrubbedarray_rad = [cs_s_rad, hc_s_rad, lc_s_rad]

# Mix both reflectance and radiance values for clear sky, high cloud, and low cloud conditions at varying 
# percentages to produce planetary reflectance spectra at 25%, 50%, and 75% cloud cover:
# 25% cloud cover
percentage = [0.75, 0.125, 0.125]                                                               # 75% clear sky & 12.5% high and low cloud
mixed25, rad25 = mixthemup(orginalarray, orginalarray_rad, percentage)                          # Call definition with biosignatures
mixed25_scrubbed, rad25_scrubbed = mixthemup(scrubbedarray, scrubbedarray_rad, percentage)      # Call definition without biosignatures

# 50% cloud cover
percentage = [0.5, 0.25, 0.25]                                                                  # 50% clear sky & 25% high and low cloud
mixed50, rad50 = mixthemup(orginalarray, orginalarray_rad, percentage)                          # Call definition with biosignatures
mixed50_scrubbed, rad50_scrubbed = mixthemup(scrubbedarray, scrubbedarray_rad, percentage)      # Call definition without biosignatures

# 75% cloud cover
percentage = [0.25, 0.375, 0.375]                                                               # 25% clear sky & 37.5% high and low cloud
mixed75, rad75 = mixthemup(orginalarray, orginalarray_rad, percentage)                          # Call defintion with biosignatures
mixed75_scrubbed, rad75_scrubbed = mixthemup(scrubbedarray, scrubbedarray_rad, percentage)      # Call definition without biosignatures

98990
Size in method: 98990
Size in method: 98990
Size of refarray: 98990
Size of radarray: 98990
size fo ref: 98990
size fo ref: 98990
size fo ref: 98990
Size in method: 98990
Size in method: 98990
Size of refarray: 98990
Size of radarray: 98990
size fo ref: 98990
size fo ref: 98990
size fo ref: 98990
Size in method: 98990
Size in method: 98990
Size of refarray: 98990
Size of radarray: 98990
size fo ref: 98990
size fo ref: 98990
size fo ref: 98990
Size in method: 98990
Size in method: 98990
Size of refarray: 98990
Size of radarray: 98990
size fo ref: 98990
size fo ref: 98990
size fo ref: 98990
Size in method: 98990
Size in method: 98990
Size of refarray: 98990
Size of radarray: 98990
size fo ref: 98990
size fo ref: 98990
size fo ref: 98990
Size in method: 98990
Size in method: 98990
Size of refarray: 98990
Size of radarray: 98990
size fo ref: 98990
size fo ref: 98990
size fo ref: 98990


In [16]:
#######################################################################################
## Write the wavelength and reflectance to a file
#######################################################################################

# Define name for saving file
name = "Arthro"       # Use name of microbe, like Arthrobacter

# Write wavelength, flux, mixed 25% reflectance, and mixed 25% radiance to a file with and without biosignatures
writetofile(name+"_mixed25.txt", wave, flux, mixed25, rad25)
writetofile(name+"_mixed25_scrubbed.txt", wave, flux, mixed25_scrubbed, rad25_scrubbed)

# Write wavelength, flux, mixed 50% reflectance, and mixed 50% radiance to a file with and without biosignatures
writetofile(name+"_mixed50.txt", wave, flux, mixed50, rad50)
writetofile(name+"_mixed50_scrubbed.txt", wave, flux, mixed50_scrubbed, rad50_scrubbed)

# Write wavelength, flux, mixed 75% reflectance, and mixed 75% radiance to a file with and without biosignatures
writetofile(name+"_mixed75.txt", wave, flux, mixed75, rad75)
writetofile(name+"_mixed75_scrubbed.txt", wave, flux, mixed75_scrubbed, rad75_scrubbed)